In [1]:
MLFLOW_TRACKING_URI = '../models/mlruns/'
MLFLOW_RUN_ID = "31ae402b0ee14d82934009728f2c3caf"

LOG_DATA_PKL    =  "data.pkl"
LOG_MODEL_PKL   =  "model.pkl"
LOG_METRICS_PKL =  "metrics.pkl"

CLUSTERS_YAML_PATH = "../data/processed/features_skills_clusters_description.yaml"

In [2]:
import os 
import sklearn
import pickle
import yaml

import pandas as pd

import mlflow
from mlflow.tracking import MlflowClient

C:\Users\MR\anaconda3\lib\site-packages\paramiko\transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.scriptrun = azureml.core.script_run:ScriptRun._from_run_dto with exception (pyOpenSSL 23.0.0 (c:\users\mr\anaconda3\lib\site-packages), Requirement.parse('pyopenssl<23.0.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (pyOpenSSL 23.0.0 (c:\users\mr\anaconda3\lib\site-packages), Requirement.parse('pyopenssl<23.0.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_reused_dto with exception (pyOpenSSL 23.0.0 (c:\users\mr\anaconda3\lib\site-packages), Requirement.parse('pyopenssl<23.0.0'), {'azureml-core'}).
Failure while l

# Initialize

# 1. Mlflow

In [3]:
# Initialize client and experiment
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
client = MlflowClient()

run = mlflow.get_run(MLFLOW_RUN_ID)
artificats_path = run.info.artifact_uri

## Load model

In [4]:
# Load model
model_path = os.path.join(artificats_path, LOG_MODEL_PKL).replace("file:///", "")
with open(model_path, "rb") as f:
    model = pickle.load(f)

model

{'model_description': 'Random Forest: with PCA + Hyperparamter tuning',
 'model_details': "GridSearchCV(estimator=Pipeline(steps=[('robustscaler', RobustScaler()),\n                                       ('pca', PCA()),\n                                       ('randomforestclassifier',\n                                        RandomForestClassifier(n_jobs=8,\n                                                               random_state=0,\n                                                               verbose=1))]),\n             param_grid=[{'pca__n_components': [0.7, 0.85, 0.95],\n                          'randomforestclassifier__max_depth': [3, 10, None],\n                          'randomforestclassifier__n_estimators': [250, 500]}])",
 'model_object': GridSearchCV(estimator=Pipeline(steps=[('robustscaler', RobustScaler()),
                                        ('pca', PCA()),
                                        ('randomforestclassifier',
                                      

In [5]:
# Load data pkl
data_path  = os.path.join(artificats_path, LOG_DATA_PKL).replace("file:///", "")
with open(data_path, 'rb') as data_file:
    data = pickle.load(data_file)

data.keys()

dict_keys(['data_path', 'training_indices', 'test_indices', 'features_names', 'targets_names'])

In [6]:
# Unpack vars
features_names = pd.Series(data["features_names"])
targets_names  = pd.Series(data['targets_names'])
classifier = model['model_object']

In [7]:
classifier

GridSearchCV(estimator=Pipeline(steps=[('robustscaler', RobustScaler()),
                                       ('pca', PCA()),
                                       ('randomforestclassifier',
                                        RandomForestClassifier(n_jobs=8,
                                                               random_state=0,
                                                               verbose=1))]),
             param_grid=[{'pca__n_components': [0.7, 0.85, 0.95],
                          'randomforestclassifier__max_depth': [3, 10, None],
                          'randomforestclassifier__n_estimators': [250, 500]}])

______

## Load skills Clusters

In [8]:
# Load skills clusters
with open(CLUSTERS_YAML_PATH, "r") as clusters_file:
    skills_clusters = yaml.safe_load(clusters_file)

skills_clusters

{'skills_group_0': ['HTML/CSS',
  'JavaScript',
  'TypeScript',
  'MongoDB',
  'Angular',
  'Angular.js',
  'Express',
  'Fastify',
  'Gatsby',
  'Next.js',
  'Node.js',
  'React.js',
  'Electron',
  'React Native',
  'Yarn',
  'npm'],
 'skills_group_1': ['Elixir', 'Erlang', 'Ruby', 'Phoenix', 'Ruby on Rails'],
 'skills_group_10': ['MATLAB',
  'Python',
  'Django',
  'FastAPI',
  'Flask',
  'NumPy',
  'Pandas',
  'Scikit-learn'],
 'skills_group_11': ['Assembly', 'C', 'C++', 'GTK', 'Qt'],
 'skills_group_12': ['Dart',
  'Cloud Firestore',
  'Firebase Realtime Database',
  'Flutter'],
 'skills_group_13': ['CouchDB', 'Couchbase'],
 'skills_group_14': ['DynamoDB',
  'Ansible',
  'Chef',
  'Pulumi',
  'Puppet',
  'Terraform'],
 'skills_group_15': ['Unity 3D', 'Unreal Engine'],
 'skills_group_16': ['Deno', 'Svelte'],
 'skills_group_17': ['Nuxt.js', 'Vue.js'],
 'skills_group_18': ['Hugging Face Transformers',
  'Keras',
  'TensorFlow',
  'Torch/PyTorch'],
 'skills_group_19': ['Capacitor', 'Cor

In [9]:
# Reformat into data frame
molten_clusters = [(cluster_name, cluster_skill)
                   for cluster_name, cluster_skills in skills_clusters.items()
                   for cluster_skill in cluster_skills]

clusters_df = pd.DataFrame(molten_clusters, columns=["cluster_name", "skill"])
clusters_df

,cluster_name,skill
0,skills_group_0,HTML/CSS
1,skills_group_0,JavaScript
2,skills_group_0,TypeScript
3,skills_group_0,MongoDB
4,skills_group_0,Angular
...,...,...
115,skills_group_8,Redis
116,skills_group_8,Apache Kafka
117,skills_group_9,R
118,skills_group_9,SAS


________

## Predict sample entry

In [10]:
sample_skills = ['NumPy', 'Pandas', 'Scikit-learn']

In [11]:
# Verify
pd.Series(sample_skills).isin(features_names)

0    True
1    True
2    True
dtype: bool

### 1. Recreate cluster features

In [12]:
sample_clusters = clusters_df.copy()
sample_clusters["sample_skills"] = sample_clusters["skill"].isin(sample_skills)
sample_clusters

,cluster_name,skill,sample_skills
0,skills_group_0,HTML/CSS,False
1,skills_group_0,JavaScript,False
2,skills_group_0,TypeScript,False
3,skills_group_0,MongoDB,False
4,skills_group_0,Angular,False
...,...,...,...
115,skills_group_8,Redis,False
116,skills_group_8,Apache Kafka,False
117,skills_group_9,R,False
118,skills_group_9,SAS,False


In [13]:
cluster_features = sample_clusters.groupby("cluster_name")["sample_skills"].sum()
cluster_features

cluster_name
skills_group_0     0
skills_group_1     0
skills_group_10    3
skills_group_11    0
skills_group_12    0
skills_group_13    0
skills_group_14    0
skills_group_15    0
skills_group_16    0
skills_group_17    0
skills_group_18    0
skills_group_19    0
skills_group_2     0
skills_group_20    0
skills_group_21    0
skills_group_3     0
skills_group_4     0
skills_group_5     0
skills_group_6     0
skills_group_7     0
skills_group_8     0
skills_group_9     0
Name: sample_skills, dtype: int64

### 2. Create OneHotEncoded skills

In [14]:
#Exclude clusters names & get only skills names
skills_names = features_names[~features_names.isin(cluster_features.index)]
skills_names

0                     APL
1                Assembly
2              Bash/Shell
3                       C
4                      C#
              ...        
156                   Vim
157         Visual Studio
158    Visual Studio Code
159              Webstorm
160                 Xcode
Length: 161, dtype: object

In [15]:
ohe_skills = pd.Series(skills_names.isin(sample_skills).astype(int).tolist(), 
                       index=skills_names)
ohe_skills

APL                   0
Assembly              0
Bash/Shell            0
C                     0
C#                    0
                     ..
Vim                   0
Visual Studio         0
Visual Studio Code    0
Webstorm              0
Xcode                 0
Length: 161, dtype: int64

### 3. Combine features

In [16]:
# Concat
features = pd.concat([ohe_skills,
                      cluster_features])
features

APL               0
Assembly          0
Bash/Shell        0
C                 0
C#                0
                 ..
skills_group_5    0
skills_group_6    0
skills_group_7    0
skills_group_8    0
skills_group_9    0
Length: 183, dtype: int64

In [17]:
# Sort columns
features = features.loc[features_names]
features

APL               0
Assembly          0
Bash/Shell        0
C                 0
C#                0
                 ..
skills_group_5    0
skills_group_6    0
skills_group_7    0
skills_group_8    0
skills_group_9    0
Length: 183, dtype: int64

### 4. Predict

In [18]:
prob_predictions = classifier.predict_proba([features.values])
prob_predictions

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished


[array([[0.62, 0.38]]),
 array([[0.996, 0.004]]),
 array([[0.99, 0.01]]),
 array([[0.804, 0.196]]),
 array([[0.765, 0.235]]),
 array([[0.996, 0.004]]),
 array([[0.996, 0.004]]),
 array([[0.982, 0.018]]),
 array([[0.91833333, 0.08166667]]),
 array([[0.972, 0.028]]),
 array([[0.986, 0.014]]),
 array([[0.991, 0.009]]),
 array([[0.952, 0.048]]),
 array([[0.99666667, 0.00333333]]),
 array([[0.986, 0.014]]),
 array([[0.86, 0.14]]),
 array([[0.95, 0.05]]),
 array([[0.704, 0.296]]),
 array([[0.978, 0.022]]),
 array([[0.996, 0.004]])]

In [19]:
positive_probs = [prob[0][1] for prob in prob_predictions]
job_prediction = pd.Series(positive_probs, 
                           index=targets_names).sort_values(ascending=False)

job_prediction

Academic researcher                              0.380000
Scientist                                        0.296000
Data scientist or machine learning specialist    0.235000
Data or business analyst                         0.196000
Engineer, data                                   0.140000
Developer, back-end                              0.081667
Project manager                                  0.050000
Developer, full-stack                            0.048000
Developer, desktop or enterprise applications    0.028000
Security professional                            0.022000
Developer, QA or test                            0.018000
Developer, mobile                                0.014000
Developer, embedded applications or devices      0.014000
Cloud infrastructure engineer                    0.010000
Developer, front-end                             0.009000
Blockchain                                       0.004000
DevOps specialist                                0.004000
Database admin